In [12]:
# load relevant libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

# Read in Greenleaf Train.csv train file
url = "/Users/Glenn/Documents/GA_Data_Science/Kaggle/SpringLeaf/train.csv"
train = pd.read_csv(url).set_index("ID")

# Read in Greenleaf Train.csv teat file
url = "/Users/Glenn/Documents/GA_Data_Science/Kaggle/SpringLeaf/test.csv"
test = pd.read_csv(url).set_index("ID")

/Applications/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [2]:
# lcd.describe()

,ID,VAR_0002,VAR_0003,VAR_0004,VAR_0006,VAR_0007,VAR_0013,VAR_0014,VAR_0015,VAR_0016,...,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,target
count,145231.000000,145231.000000,145231.000000,145231.000000,145175.000000,145175.00000,145175.000000,145175.000000,145175.000000,145175.000000,...,145231.000000,145231.000000,145231.000000,145231.000000,1.452310e+05,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000
mean,145124.981808,105.278040,88.492285,3721.946210,0.693907,0.51316,0.693907,0.494493,0.476763,1.447598,...,0.549965,86.664369,89.828762,914.417259,9.904497e+08,922.938505,936.487905,9942.162279,8849.839111,0.232547
std,83890.290627,143.351237,125.903323,12280.012213,1.604578,1.17531,1.604578,1.115085,1.000755,1.132784,...,6.361312,31.347700,26.928221,276.361514,9.725818e+07,212.087416,210.370833,743.153332,3175.797333,0.422457
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000
25%,72342.500000,24.000000,13.000000,1790.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,...,0.000000,98.000000,98.000000,998.000000,1.000000e+09,998.000000,998.000000,9998.000000,9998.000000,0.000000
50%,145272.000000,60.000000,55.000000,2500.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,...,0.000000,98.000000,98.000000,998.000000,1.000000e+09,998.000000,998.000000,9998.000000,9998.000000,0.000000
75%,217686.500000,132.000000,120.000000,3600.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,0.000000,98.000000,98.000000,998.000000,1.000000e+09,998.000000,998.000000,9998.000000,9998.000000,0.000000
max,290463.000000,999.000000,999.000000,2200000.000000,68.000000,66.00000,68.000000,67.000000,35.000000,39.000000,...,99.000000,99.000000,99.000000,999.000000,1.000000e+09,999.000000,999.000000,9999.000000,9999.000000,1.000000


In [6]:
# url1 = "/Users/Glenn/Documents/GA_Data_Science/Kaggle/GreenLeaf/train_describe.txt"
# f = open(url1, 'w')
# f.write(str(lcd.describe()))

In [7]:
# f.close()

In [14]:
# remove constants
nunique = pd.Series([train[col].nunique() for col in train.columns], index = train.columns)
constants = nunique[nunique<2].index.tolist()
train = train.drop(constants,axis=1)

In [16]:
from sklearn import ensemble, preprocessing, cross_validation
from sklearn.metrics import roc_auc_score as auc
from time import time

# encode string
strings = train.dtypes == 'object'; strings = strings[strings].index.tolist(); encoders = {}
for col in strings:
    encoders[col] = preprocessing.LabelEncoder()
    train[col] = encoders[col].fit_transform(train[col])
    try:
        test[col] = encoders[col].transform(test[col])
    except:
        # lazy way to incorporate the feature only if can be encoded in the test set
        del test[col]
        del train[col]



/Applications/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:198: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/Applications/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:251: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]
/Applications/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:384: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  bool_ar = (sar[1:] == sar[:-1])


In [18]:
# DATA ready
X = train.drop('target',1).fillna(0); y = train.target

# RF FTW :)
rf = ensemble.RandomForestClassifier(n_jobs=4, n_estimators = 20, random_state = 11)

In [19]:
# CROSS VALIDATE AND PRINT TRAIN AND TEST SCORE
kf = cross_validation.StratifiedKFold(y, n_folds=3, shuffle=True, random_state=11)
trscores, cvscores, times = [], [], []
for itr, icv in kf:
    t = time()
    trscore = auc(y.iloc[itr], rf.fit(X.iloc[itr], y.iloc[itr]).predict_proba(X.iloc[itr])[:,1])
    cvscore = auc(y.iloc[icv], rf.predict_proba(X.iloc[icv])[:,1])
    trscores.append(trscore); cvscores.append(cvscore); times.append(time()-t)
print "TRAIN %.4f | TEST %.4f | TIME %.2fm (1-fold)" % (np.mean(trscores), np.mean(cvscores), np.mean(times)/60)

TRAIN 1.0000 | TEST 0.7297 | TIME 7.91m (1-fold)
